# Example: Read 1 Degree Sea Ice (CICE6) Replay Data, Store to Zar

Example using [CICE6Dataset](generated/ufs2arco.CICE6Dataset.rst) to read data from two cycles of the replay dataset.

In [1]:
from collections.abc import Iterable

import xarray as xr
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append("..")
from ufs2arco import CICE6Dataset

## Setup path to read from

In this case, read replay data from the s3 bucket.
Right now, [CICE6Dataset](generated/ufs2arco.CICE6Dataset.rst) requires a `path_in` callable with 3 entries, so that it can build file paths for a given date (denoting a DA cycle), forecast hours to grab, and file prefixes.

In [3]:
def seaice_path(cycles, forecast_hours, file_prefixes):

    upper = "s3://noaa-ufs-gefsv13replay-pds/1deg"
    cycles = [cycles] if not isinstance(cycles, Iterable) else cycles

    files = []
    for cycle in cycles:
        this_dir = f"{cycle.year:04d}/{cycle.month:02d}/{cycle.year:04d}{cycle.month:02d}{cycle.day:02d}{cycle.hour:02d}"
        for fp in file_prefixes:
            for fhr in forecast_hours:
                fhr_delta = timedelta(hours=-9+fhr)
                this_date = cycle+fhr_delta
                this_file = f"{this_dir}/{fp}{this_date.year:04d}-{this_date.month:02d}-{this_date.day:02d}-{this_date.hour*3600:d}.nc"
                files.append(this_file)
    return [f"{upper}/{this_file}" for this_file in files]

For instance to grab the `ocn_` files at forecast hour `fhr00` 
from the DA cycle at 00:00 Jan 1, 1994, this would be used as follows:

In [4]:
cycles = [datetime(1994,1,1,0), datetime(1994,1,1,12)]
seaice_path(cycles, [0], ["iceh_ic."])

['s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/iceh_ic.1993-12-31-54000.nc',
 's3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010112/iceh_ic.1994-01-01-10800.nc']

However, reading from s3 is super slow with `xarray.open_dataset`.
Luckily we can tell fsspec to cache the files locally before opening, this is denoted by
prepending the names as follows.

See discussion [here](https://discourse.pangeo.io/t/reading-goes-r-s3-netcdfs-from-an-aws-ec2-instance-is-it-possible-to-get-faster-speeds-than-from-my-local-machine/2440/13)
for more info.

In [5]:
def cached_path(cycles, forecast_hours, file_prefixes):
    return [f"simplecache::{u}" for u in seaice_path(cycles, forecast_hours, file_prefixes)]

In [6]:
cached_path(cycles, [0], ["iceh_ic."])

['simplecache::s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/iceh_ic.1993-12-31-54000.nc',
 'simplecache::s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010112/iceh_ic.1994-01-01-10800.nc']

## Use CICE6Dataset

Some of that is wrapped up under the hood in [CICE6Dataset](generated/ufs2arco.CICE6Dataset.rst), we just need to give two inputs:

1. The filename mapping as defined above

2. A configuration yaml file

In [7]:
reader = CICE6Dataset(path_in=cached_path, config_filename="config-replay.yaml")

CICE6Dataset.__init__: Could not find coords_path_out in config-replay.yaml, using default.


We'll use this reader to open a dataset with [.open_dataset()](generated/ufs2arco.MOM6Dataset.open_dataset.rst), note that the `fsspec_kwargs` is not necessary for local file reads, but necessary for reading from buckets like s3

In [32]:
%%time
ds = reader.open_dataset(cycles, fsspec_kwargs={"s3":{"anon":True}})
ds

CPU times: user 374 ms, sys: 53.4 ms, total: 428 ms
Wall time: 392 ms


<xarray.Dataset>
Dimensions:         (time: 2, nj: 320, ni: 360, nc: 5, nkaer: 5)
Coordinates:
  * time            (time) datetime64[ns] 1993-12-31T15:00:00 1994-01-01T03:0...
    TLON            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    TLAT            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ULON            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ULAT            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    NLON            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    NLAT            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ELON            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ELAT            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
Dimensions without coordinates: nj, ni, nc, nkaer
Data variables: (12/194)
    NCAT            (time, nc) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    VGRDa           (time, nkaer) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    tmask           (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    umask           (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    nmask           (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    emask           (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    ...              ...
    ipond_d         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    apeff_d         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    apond_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    hpond_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    ipond_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    apeff_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
Attributes:
    title:        sea ice model output for CICE
    contents:     Diagnostic and Prognostic Variables
    source:       Los Alamos Sea Ice Model, CICE_6.0.2
    comment:      This year has 365 days
    comment2:     File written on model date 19931231
    comment3:     seconds elapsed into model date:  54000
    conventions:  CF-1.0
    history:      This dataset was created on 2023-08-15 at 10:41:27.1
    io_flavor:    io_netcdf

In [26]:
ds["cftime"] = reader._time2cftime(ds["time"])

In [27]:
ds["ftime"] = reader._time2ftime(ds["time"], cycles)

In [31]:
ds.NCAT.values

array([[0.6445072, 1.3914335, 2.4701793, 4.567288 , 9.333842 ],
       [0.6445072, 1.3914335, 2.4701793, 4.567288 , 9.333842 ]],
      dtype=float32)

In [29]:
swap = {
    "nc": "NCAT",
    "nkaer": "VGRDa",
    "nkice": "VGRDi",
    "nksnow": "VGRDs",
    "d2": "time_bounds",
}

In [33]:
ds

<xarray.Dataset>
Dimensions:         (time: 2, nj: 320, ni: 360, nc: 5, nkaer: 5)
Coordinates:
  * time            (time) datetime64[ns] 1993-12-31T15:00:00 1994-01-01T03:0...
    TLON            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    TLAT            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ULON            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ULAT            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    NLON            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    NLAT            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ELON            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
    ELAT            (nj, ni) float32 dask.array<chunksize=(320, 360), meta=np.ndarray>
Dimensions without coordinates: nj, ni, nc, nkaer
Data variables: (12/194)
    NCAT            (time, nc) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    VGRDa           (time, nkaer) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    tmask           (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    umask           (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    nmask           (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    emask           (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    ...              ...
    ipond_d         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    apeff_d         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    apond_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    hpond_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    ipond_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    apeff_h         (time, nj, ni) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
Attributes:
    title:        sea ice model output for CICE
    contents:     Diagnostic and Prognostic Variables
    source:       Los Alamos Sea Ice Model, CICE_6.0.2
    comment:      This year has 365 days
    comment2:     File written on model date 19931231
    comment3:     seconds elapsed into model date:  54000
    conventions:  CF-1.0
    history:      This dataset was created on 2023-08-15 at 10:41:27.1
    io_flavor:    io_netcdf

In [30]:
for org, new in swap.items():
    if new in xds:
        print(new)
        print(xds[new].values)
        if "time" in xds[new].dims:
            xds[new] = xds[new].isel(time=0)
        print(xds[new].values)
        xds = xds.swap_dims({org:new})
    if org in xds:
        xds = xds.drop(org)

    print()
        

NCAT
[0.6445072 1.3914335 2.4701793 4.567288  9.333842 ]
[0.6445072 1.3914335 2.4701793 4.567288  9.333842 ]


ValueError: cannot swap from dimension 'nc' because it is not one of the dimensions of this dataset ('time', 'nj', 'ni', 'NCAT', 'VGRDa')

xarray gives a nice view of the data, we can look at a single variable to see how the data are laid out

In [9]:
ds["temp"]

<xarray.DataArray 'temp' (time: 2, z_l: 75, yh: 320, xh: 360)>
dask.array<concatenate, shape=(2, 75, 320, 360), dtype=float32, chunksize=(1, 1, 320, 360), chunktype=numpy.ndarray>
Coordinates:
  * yh       (yh) float64 -77.77 -77.56 -77.34 -77.12 ... 89.26 89.48 89.69 89.9
  * z_l      (z_l) float64 0.5154 1.571 2.687 ... 5.495e+03 5.698e+03 5.902e+03
    cftime   (time) object 1994-01-01 00:00:00 1994-01-01 06:00:00
  * xh       (xh) float64 -299.5 -298.5 -297.5 -296.5 ... 56.5 57.5 58.5 59.5
  * time     (time) datetime64[ns] 1994-01-01 1994-01-01T06:00:00
    ftime    (time) timedelta64[ns] 00:00:00 00:00:00
Attributes:
    units:          degC
    long_name:      Potential Temperature
    cell_methods:   area:mean z_l:mean yh:mean xh:mean time: mean
    time_avg_info:  average_T1,average_T2,average_DT

## Storing the dataset

The dataset has more variables than we wanted, but these are trimmed when we call [.store_dataset()](generated/ufs2arco.MOM6Dataset.store_dataset.rst).

Any arguments beyond the dataset are passed to `xarray.to_zarr`.

By default, dask (which we're accessing through xarray under the hood) stores data to zarr using the chunksizes that are being used currently by dask (i.e., how the data are viewed in memory, currently).
However, we will change this layout based on the `chunks_out` option in our configuration yaml.

In [10]:
reader.chunks_out

{'time': 1, 'z_l': -1, 'z_i': -1, 'yh': 60, 'xh': 60, 'yq': 60, 'xq': 60}

In [11]:
reader.store_dataset(ds, mode="w")

Stored dataset at example-replay-1deg/mom6.zarr


In [12]:
xds = xr.open_zarr(reader.data_path)

In [13]:
xds

<xarray.Dataset>
Dimensions:  (time: 2, yh: 320, xh: 360, z_l: 75, xq: 360, yq: 320)
Coordinates:
    cftime   (time) object dask.array<chunksize=(2,), meta=np.ndarray>
    ftime    (time) timedelta64[ns] dask.array<chunksize=(2,), meta=np.ndarray>
  * time     (time) datetime64[ns] 1994-01-01 1994-01-01T06:00:00
  * xh       (xh) float64 -299.5 -298.5 -297.5 -296.5 ... 56.5 57.5 58.5 59.5
  * xq       (xq) float64 -299.0 -298.0 -297.0 -296.0 ... 57.0 58.0 59.0 60.0
  * yh       (yh) float64 -77.77 -77.56 -77.34 -77.12 ... 89.26 89.48 89.69 89.9
  * yq       (yq) float64 -77.67 -77.45 -77.23 -77.01 ... 89.37 89.59 89.8 90.0
  * z_l      (z_l) float64 0.5154 1.571 2.687 ... 5.495e+03 5.698e+03 5.902e+03
Data variables:
    SSH      (time, yh, xh) float32 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
    so       (time, z_l, yh, xh) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
    temp     (time, z_l, yh, xh) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
    uo       (time, z_l, yh, xq) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
    vo       (time, z_l, xh, yq) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
Attributes:
    NumFilesInSet:  1
    grid_tile:      N/A
    grid_type:      regular
    title:          UFS

We can see that this only has the data variables we asked for in the config yaml file.

In [14]:
xds.temp

<xarray.DataArray 'temp' (time: 2, z_l: 75, yh: 320, xh: 360)>
dask.array<open_dataset-temp, shape=(2, 75, 320, 360), dtype=float32, chunksize=(1, 75, 60, 60), chunktype=numpy.ndarray>
Coordinates:
    cftime   (time) object dask.array<chunksize=(2,), meta=np.ndarray>
    ftime    (time) timedelta64[ns] dask.array<chunksize=(2,), meta=np.ndarray>
  * time     (time) datetime64[ns] 1994-01-01 1994-01-01T06:00:00
  * xh       (xh) float64 -299.5 -298.5 -297.5 -296.5 ... 56.5 57.5 58.5 59.5
  * yh       (yh) float64 -77.77 -77.56 -77.34 -77.12 ... 89.26 89.48 89.69 89.9
  * z_l      (z_l) float64 0.5154 1.571 2.687 ... 5.495e+03 5.698e+03 5.902e+03
Attributes:
    cell_methods:   area:mean z_l:mean yh:mean xh:mean time: mean
    long_name:      Potential Temperature
    time_avg_info:  average_T1,average_T2,average_DT
    units:          degC

And we can see this has a different chunking scheme than the original dataset, it has what we asked for in the config yaml.

In [15]:
reader.chunks_out

{'time': 1, 'z_l': -1, 'z_i': -1, 'yh': 60, 'xh': 60, 'yq': 60, 'xq': 60}

<div class="alert alert-info">

Note:

This chunking scheme is not necessarily desirable, as the chunk sizes are quite small. This is simply used as an example to illustrate that we have control
over the chunk sizes.

</div>

## Cleanup

In [16]:
from shutil import rmtree

In [17]:
rmtree(reader.data_path)